# Reverse engineer the rank algorithm

Adapted from [calculateRank.js][1]

[1]: https://github.com/anuraghazra/github-readme-stats/blob/f83080601e87dfe76b003cd08c675cbb2f6204e9/src/calculateRank.js#L2

### Define constants

In [1]:
offset = Dict([("commits", 1.65),
               ("contribs", 1.65),
               ("issues", 1),
               ("stars", 0.75),
               ("prs", 0.5),
               ("followers", 0.45),
               ("repos", 1)])

Dict{String, Real} with 7 entries:
  "stars"     => 0.75
  "prs"       => 0.5
  "repos"     => 1
  "issues"    => 1
  "commits"   => 1.65
  "followers" => 0.45
  "contribs"  => 1.65

In [2]:
@enum Rank begin
    s = 1
    a1 = 25
    a2 = 45
    a3 = 60
    b = 100
end

### Define helper function

In [3]:
function normalcdf(μ, σ, ∑)
    z = (∑ - μ) / sqrt(2 * σ * σ) 
    t = 1 / (1 + 0.3275911 * abs(z))
    w = (0.254829592, -0.284496736, 1.421413741, -1.453152027, 1.061405429)
    erf = 1 - ((((w[5] * t + w[4]) * t + w[3]) * t + w[2]) * t + w[1]) * t * exp(-z * z)
    sign = z < 0 ? -1 : 1
    (1 + sign * erf) / 2
end

normalcdf (generic function with 1 method)

### Define compute functions

In [4]:
function compute_score(stats)
    @assert keys(offset) == keys(stats)
    score = 0
    for key in keys(offset)
        score += offset[key] * stats[key]
    end
    score / 100
end

compute_score (generic function with 1 method)

In [5]:
function compute_normalized_score(score)
    total_rank = sum(Integer, [s, a1, a2, a3, b])
    total_offset = sum(values(offset))
    normalized = normalcdf(score, total_rank, total_offset)
    normalized * 100
end

compute_normalized_score(0)

51.208736359772786

In [6]:
function compute_rank(normalized_score)
    @assert normalized_score >= 0
    if normalized_score < Integer(s)
        level = "S+"
    elseif normalized_score < Integer(a1)
        level = "S"
    elseif normalized_score < Integer(a2)
        level = "A++"
    elseif normalized_score < Integer(a3)
        level = "A+"
    elseif normalized_score < Integer(b)
        level = "B+"
    else
        level = "B"
    end
    level
end

compute_rank(51.21)

"A+"

### Sanity check with my GitHub stats

In [7]:
my_stats = Dict([("repos", 33),
                 ("commits", 880),
                 ("contribs", 7),
                 ("followers", 5),
                 ("prs", 30),
                 ("issues", 41),
                 ("stars", 1)])
my_score = compute_score(my_stats)
my_normalized_score = compute_normalized_score(my_score)
my_rank = compute_rank(my_normalized_score)
println("score = $my_score rank = $my_rank")

score = 15.5555 rank = A+


### Everyone gets an A+!

In [8]:
zero_normalized_score = compute_normalized_score(0)
zero_rank = compute_rank(zero_normalized_score)
zero_rank

"A+"

### Find the rank distribution

We can perform the inverse of normalcdf to find the target in constant time, but for now, we will look for it using binary search

In [9]:
function binary_search(func, target, lower=-1, upper=1e3)
    mid = -1
    while lower <= upper
        mid = (lower + upper) ÷ 2
        if func(mid) < target
            upper = mid - 1
        else
            lower = mid + 1
        end
    end
    lower
end

find_threshold(x) = binary_search(compute_normalized_score, x)

find_threshold (generic function with 1 method)

In [10]:
names = ("A++", "S", "S+")
ranks = (a2, a1, s)
thresholds = Dict()
for (name, rank) in zip(names, ranks)
    threshold = find_threshold(Integer(rank))
    println("score need to reach $name = $threshold")
    thresholds[name] = threshold
end

score need to reach A++ = 37.0
score need to reach S = 163.0
score need to reach S+ = 545.0


### How many commits until I reach the next rank?

In [11]:
function calculate_commmits(current_score, next_rank)
    @assert current_score < next_rank
    gap = (next_rank - current_score) * 100
    commits = round(gap / offset["commits"])
    daily = round(commits / 365, digits=1)
    println("You need $commits commits, or around ",
            "$daily commits a day for the next 365 days.")
end

calculate_commmits(my_score, thresholds["A++"])

You need 1300.0 commits, or around 3.6 commits a day for the next 365 days.
